<a href="https://colab.research.google.com/github/Alessandro1103/ML/blob/main/MLEx16_CNNEnsembles_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Exercise 16

# CNN Ensembles - CIFAR10



##Import

Import libraries and print some versions.

To use GPU, set `Edit / Notebook settings / Hardware accelerator` to **GPU**.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random

print("Tensorflow version %s" %tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Tensorflow version 2.15.0


SystemError: ignored

## Load data

Load training data from Keras library




In [ ]:
def load_cifar10():
    # load data
    (Xtrain,Ytrain), (Xtest, Ytest) = tf.keras.datasets.cifar10.load_data()
    print(Ytrain.shape)
    # get information
    ninput = Xtrain.shape[0]
    imgsize = (Xtrain.shape[1], Xtrain.shape[2])
    input_shape = (Xtrain.shape[1], Xtrain.shape[2], Xtrain.shape[3])
    ntest = Xtest.shape[0]
    num_classes = np.max(np.unique(Ytrain)) + 1
    print("Training input %s" %str(Xtrain.shape))
    print("Training output %s" %str(Ytrain.shape))
    print("Test input %s" %str(Xtest.shape))
    print("Test output %s" %str(Ytest.shape))
    print("Input shape: %s" %str(input_shape))
    print("Number of classes: %d" %num_classes)

    # normalize input to [0,1]
    Xtrain = Xtrain / 255.0
    Xtest = Xtest / 255.0

    # Transform output to one-out-of-n encodingmembers
    Ytrain = tf.keras.utils.to_categorical(Ytrain, num_classes)
    Ytest = tf.keras.utils.to_categorical(Ytest, num_classes)

    return [Xtrain,Ytrain,Xtest,Ytest,input_shape,num_classes]

[Xtrain,Ytrain,Xtest,Ytest,input_shape,num_classes] = load_cifar10()

## Show random image



In [ ]:
i = random.randrange(0,Xtrain.shape[0])
image = Xtrain[i]
image = np.array(image, dtype='float')

label = Ytrain[i].argmax()  # categorical from one-hot-encoding
print(label)

plt.imshow(image, cmap='gray')
plt.show()


##CNN ensemble model


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,\
                         Conv2D, MaxPool2D, Input, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers

# single model
def CNN(input_shape, num_classes, base_width=16, depth=4):

    inputs = Input(input_shape)

    image_size = input_shape[0]
    filters = base_width
    kernel_size = 3
    # feature extractor
    for i in range(depth):
        if i == 0:
            x = Conv2D(filters=filters,
                       kernel_size = kernel_size,
                       activation="relu",
                       strides=1,
                       padding="same")(inputs)
        else:
            x = Conv2D(filters=filters,
                       kernel_size = kernel_size,
                       activation="relu",
                       strides=1,
                       padding="same")(x)
        x = MaxPool2D(pool_size=(2,2), strides=(2,2), padding="valid")(x)
        filters *= 2

    x = Flatten()(x)
    x = Dropout(0.4)(x)

    outputs = Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# ensemble model
def Ensemble(ens_dim, input_shape, num_classes, base_width=16, depth=4):
  inputs = Input(shape=input_shape)

  learners = [CNN(input_shape,
                  num_classes,
                  base_width,
                  depth) for _ in range(ens_dim)]

  outputs = [learners[i](inputs) for i in range(ens_dim)]

  ensemble_model = Model(inputs, outputs, name="ensemble_model")
  optimizer = 'adam'
  # an independent loss for each model
  losses = ["categorical_crossentropy" for _ in range(ens_dim)]
  ensemble_model.compile(loss=losses, optimizer=optimizer, metrics=['accuracy'])
  return ensemble_model


# create the ensemble model
ens_dim = 3
model = Ensemble(ens_dim, input_shape, num_classes)
model.summary()

# single model summary
model.get_layer(index=-1).summary()

# Train setup

In [ ]:
import sklearn.metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def test_accuracy(model, history):
    # accuracy
    ind_test_acc = []
    for key in history.history.keys():
      if ("val" in key) and ("accuracy" in key):
        ind_test_acc.append(history.history[key][-1])
    print("Single models test accuracy: ", ind_test_acc)

    # ensemble test accuracy
    preds = np.array(model.predict(Xtest))
    # average over the ensemble
    preds_ens = np.mean(preds, axis=0)
    ens_test_acc = accuracy_score(np.argmax(Ytest, axis=-1), np.argmax(preds_ens, axis=-1))
    print("Ensemble test accuracy: ", ens_test_acc)

    return ind_test_acc, ens_test_acc


history = tf.keras.callbacks.History()
ind_test_accuracy = []
ens_test_accuracy = []

## Train

In [ ]:
epochs = 5
for _ in range(epochs):
    model.fit(Xtrain, [Ytrain for _ in range(ens_dim)], batch_size=32, epochs=1, callbacks=[history], validation_data = (Xtest,[Ytest for _ in range(ens_dim)]))

    iacc, eacc = test_accuracy(model, history)
    ind_test_accuracy.append(iacc)
    ens_test_accuracy.append(eacc)


##Evaluate the model

## Compare single networks with ensemble


In [ ]:
print(ind_test_accuracy)
print(ens_test_accuracy)


In [ ]:
plt.plot(ind_test_accuracy, label="Single network")
plt.plot(ens_test_accuracy, linestyle="--", label="Ensemble of {} nets".format(ens_dim))
plt.legend()
plt.ylabel('test accuracy')
plt.xlabel('epoch')

# Home Exercises

**Question 1**

Evaluate the performance of ensembles varying the number of base members.